In [1]:
from utils.Dataloader import load_movies,load_ratings,load_users
movies_df = load_movies('datasets/')
ratings_df = load_ratings('datasets/')
users_df = load_users('datasets/')
display(movies_df)
display(ratings_df)

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [14]:
sum(ratings_df[ratings_df['userId'] == user_id]['movieId']==29)

1

In [ ]:
import pandas as pd
# 영화 정보를 시청 내역과 매핑하여 출력
sorted_ratings_df = ratings_df.sort_values(by=['userId', 'timestamp'])

for user_id, user_data in sorted_ratings_df.groupby('userId'):
    print(f"User {user_id}'s Movie Watching History:")
    for index, row in user_data.iterrows():
        movie_id = row['movieId']
        movie_info = movies_df[movies_df['movieId'] == movie_id]
        if not movie_info.empty:
            title = movie_info['title'].values[0]
            genres = movie_info['genres'].values[0]
            timestamp = row['timestamp']
            print(f"Movie Title: {title}, Genres: {genres}, Timestamp: {timestamp}")
    print()


In [2]:
from utils.Dataloader import load_movies,load_ratings,load_users
import numpy as np
import pandas as pd

# 예시 데이터를 생성하거나 불러옵니다.
ratings_df = load_ratings('datasets/')  # 사용자별 영화 평점 데이터
movies_df = load_movies('datasets/')    # 영화 정보 데이터
ratings_df['movieId']

0          1193
1           661
2           914
3          3408
4          2355
           ... 
1000204    1091
1000205    1094
1000206     562
1000207    1096
1000208    1097
Name: movieId, Length: 1000209, dtype: int64

In [14]:
len(ratings_df.drop_duplicates(subset='userId'))

6040

In [18]:
len(ratings_df['movieId'])

1000209

In [16]:
len(ratings_df.drop_duplicates(subset='movieId'))

3706

In [15]:
6040 3416 89247793

SyntaxError: invalid syntax (<ipython-input-15-e836b2575c24>, line 1)

In [ ]:
from utils.Dataloader import load_movies,load_ratings,load_users
import numpy as np
import pandas as pd
import tensorflow as tf
from SASRec_dir.util import *
import joblib

# 예시 데이터를 생성하거나 불러옵니다.
ratings_df = load_ratings('datasets/')  # 사용자별 영화 평점 데이터
movies_df = load_movies('datasets/')    # 영화 정보 데이터

# 사용자 ID
userId = 1

# 사용자의 상호작용 이력 (영화 ID 순서열)
seq = np.array(ratings_df[ratings_df['userId'] == userId]['movieId'].to_list())

# 사용자가 시청한 영화의 시청 시간을 추출합니다
watch_times = np.array(ratings_df[ratings_df['userId'] == userId]['timestamp'].to_list())

# 시간 순서대로 정렬된 영화 시청 시간의 배열 생성
time_seq = np.sort(watch_times)
time_span = 256
# 시간 간격 행렬 계산
time_matrix = computeRePos(time_seq, time_span)

# 추천 대상 영화 목록을 생성합니다
all_movie_ids = movies_df['movieId'].to_numpy()
item_idx = np.setdiff1d(all_movie_ids, seq)

# 모델 로드
loaded_model = joblib.load("./models/SASRec.joblib")

# 모델 사용하여 예측 수행
with tf.Session() as sess:
    predictions = loaded_model.predict(sess, userId, seq, time_matrix, item_idx)

# 예측 점수를 기준으로 정렬하고 추천 영화 순서 획득
recommended_movie_indices = np.argsort(predictions)[::-1]  # 내림차순으로 정렬
recommended_movie_ids = item_idx[recommended_movie_indices]

# 추천 영화 ID 출력
print("추천 영화 ID:", recommended_movie_ids)

c:\Users\user\anaconda3\envs\test_tp2\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\Users\user\anaconda3\envs\test_tp2\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\Users\user\anaconda3\envs\test_tp2\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\Users\user\anaconda3\envs\test_tp2\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-e248cf67bb52>", line 32, in <module>
    loaded_model = joblib.load("./models/SASRec.joblib")
  File "c:\Users\user\anaconda3\envs\test_tp2\lib\site-packages\joblib\numpy_pickle.py", line 587, in load
    obj = _unpickle(fobj, filename, mmap_mode)
  File "c:\Users\user\anaconda3\envs\test_tp2\lib\site-packages\joblib\numpy_pickle.py", line 506, in _unpickle
    obj = unpickler.load()
  File "c:\Users\user\anaconda3\envs\test_tp2\lib\pickle.py", line 1050, in load
    dispatch[key[0]](self)
  File "c:\Users\user\anaconda3\envs\test_tp2\lib\pickle.py", line 1338, in load_global
    klass = self.find_class(module, name)
  File "c:\Users\user\anaconda3\envs\test_tp2\lib\pickle.py", line 1388, in find_class
    __import__(module, level=0)
ModuleNotFoundError:

In [ ]:
!python ./SASRec/main.py --dataset=ratings --train_dir=default

Traceback (most recent call last):
  File "./SASRec/main.py", line 5, in <module>
    from SASRec.model import Model
ModuleNotFoundError: No module named 'SASRec'


In [ ]:
# 모델 불러오기
saver.restore(sess, checkpoint_path)
print(f'Model restored from {checkpoint_path}')

In [ ]:
import joblib

model = joblib.load("./SASRec_dir/SAS_model.joblib")

ModuleNotFoundError: No module named 'model'

In [ ]:
from SASRec_dir.model import Model
from utils.Dataloader import load_movies,load_ratings,load_users
import numpy as np
import pandas as pd
import tensorflow as tf
from SASRec_dir.util import *

# 예시 데이터를 생성하거나 불러옵니다.
ratings_df = load_ratings('datasets/')  # 사용자별 영화 평점 데이터
movies_df = load_movies('datasets/')    # 영화 정보 데이터

model = Model("./SASRec/SAS_model.joblib")

# 사용자 ID
userId = 1

recommended_movies = model.rec_pre(user_id, ratings_df, movies_df)
# 추천 영화 ID 출력
print("추천 영화 ID:", recommended_movies)